**Don't know what this is used for - may be an early version**
## Make Profile summaries using Jinja2 and Python Modele

*USE Python 3.7 to maintain order of Json files*

- Fetch SD file from IG
- Spec_internals from IG Package.tgz
- Create and use import Title map from ?Package?
- Transform to Python model
- use Jinja2 template to create a summary markdown file
- save markdown file



*Note need a successful build to generate since based on ig output local file
alternatively use package.files to generate*

### import python modules including R4 fhirclient models

In [ ]:
from fhirclient.r4models import structuredefinition as SD
from fhirclient.r4models import narrative as N
from fhirclient.r4models import valueset as VS
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load
from pprint import pprint
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown
import title_map as tm
import os
from stringcase import snakecase, titlecase

### Get file and return as dict

In [ ]:

def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}', encoding="utf8") as f:
        r = f.read()
        return(loads(r))
   

### Write to file

In [ ]:
def write_file(f_name, data): # write file
    #path = ''
    path = '/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes'
    with open(os.path.join(path,f_name), 'w') as f:
        f.write(data)

### Get spec_internal from package.tgz a json file which includes canonical to local relative page links

In [ ]:
import tarfile
from pprint import pprint
def get_si(path):
    with tarfile.open(name=os.path.join(path,'package.tgz'), mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('other/spec.internals')
        r = f.read()
        return(loads(r))
        

si = get_si(in_path)
path_map = si['paths']

c = 'http://hl7.org/fhir/us/core/ValueSet/birthsex'


path_map[c]

### Loop through profiles

In [ ]:
#in_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/output/StructureDefinition-'
#f_name = 'us-core-patient'
#in_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package/' # package file keeps disappearing
in_path = '//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/output'  # use local build dir.
files = [x for x in os.listdir(in_path) if x.startswith("StructureDefinition") and x.endswith('json')]
files

In [ ]:
summ_elements =[
       'label',
       'short',
       'min',
       'max',
       'type',
       'binding',
        ]

choice_types = {'valueQuantity': 'value[x]',
                'valueCodeableConcept': 'value[x]',
                'valueString': 'value[x]',
                'valueInteger': 'value[x]',
                'valueDecimal': 'value[x]',
                'valueDateTime': 'value[x]',
                'valueRange': 'value[x]',
                'valuePeriod': 'value[x]',
                'effectivedateTime': 'effective[x]',
                'effectivePeriod': 'effective[x]',
               } 

In [ ]:
for i in files:
    constraints = {}
    sd_dict = open_file(in_path,i)
    sd = SD.StructureDefinition(sd_dict)
    profile_id = sd.id
    if sd.type != 'Extension':
        print(f'{sd.url}\t{sd.title}\tSHALL\t{sd.type}')

In [ ]:
    #print(f'========{profile_id}=======')
    for i in sd.differential.element:
        path = i.path
        #print(f'====Path = {path} =====')
        # GET Invariant Dict  path: human readable invariant list.
        try:
            k = next((k for k in sd.snapshot.element if k.path == path))
            constraint = [j.human for j in k.constraint if 'dom-' not in j.key and 'ele-' not in j.key  and 'ext-' not in j.key]
        except TypeError:
            constraint = []
        except StopIteration:
            constraint = []
        if constraint:
            constraints[path]=constraint         

        for k in summ_elements:
            #print(f'differential = {path}.{k} = {getattr(i,k)}')
            if getattr(i,k) == None:
                try:
                    snap_element = (s for s in sd.snapshot.element if s.path == path)           
                    new_val = getattr(next(snap_element),k)
                    #print(f'snapshot = {path}.{k} = {new_val}')
                    setattr(i,k,new_val)
                except StopIteration: # assume is an choice data type
                    print(f'no snapshot element for {path}.{k} = {getattr(i,k)} assume is an choice data type')
                    new_plist = []
                    for p in path.split('.'):
                        try:
                            new_plist.append(choice_types[p])
                        except KeyError:
                            new_plist.append(p)
                    new_path = '.'.join(new_plist)
                    print(path,new_path)
                    snap_element = (s for s in sd.snapshot.element if s.path == new_path)           
                    new_val = getattr(next(snap_element),k)
                    #print(f'snapshot = {path}.{k} = {new_val}')
                    setattr(i,k,new_val)
            #print(f'differential post if = {path}.{k} = {getattr(i,k)}')
    summ_file = get_summary(profile_id,sd.differential.element,constraints)
    summ_file = os.linesep.join([s for s in summ_file.splitlines() if s]) # remove empty lines
    summ_file = summ_file.replace('####', '\n####')  #add a line before Must Supports
    

    f_name = f'{profile_id}-summary.md'
    write_file(f_name,summ_file)    